<a href="https://colab.research.google.com/github/Flychuban/Malaria-Prediction-/blob/main/MalariaCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
num_imgs = len(os.listdir('/content/drive/MyDrive/MalariaPrediction/cell_images/Test/Parasitized'))
num_imgs

2856

In [14]:
TARGET_SIZE = [128, 128]

In [24]:
datagen = ImageDataGenerator(rescale=1/255.0)

In [25]:
train_data = datagen.flow_from_directory("/content/drive/MyDrive/MalariaPrediction/cell_images/Train",
                                         target_size=TARGET_SIZE,
                                         class_mode = 'binary',
                                         batch_size = 16,
                                         subset='training')

Found 21846 images belonging to 2 classes.


In [28]:
val_data = datagen.flow_from_directory('/content/drive/MyDrive/MalariaPrediction/cell_images/Test',
                                       target_size=TARGET_SIZE,
                                       class_mode='binary',
                                       batch_size=16)

Found 5712 images belonging to 2 classes.


Build model

In [39]:
model = Sequential()

In [40]:
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 16)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 30, 30, 16)        0

In [45]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
early_stop_flag = EarlyStopping(monitor='val_loss', patience=2)

In [47]:
model.fit(train_data,
          validation_data=val_data,
          epochs=20,
          steps_per_epoch=len(train_data),
          validation_steps=len(val_data),
          callbacks=[early_stop_flag])

Epoch 1/20
1366/1366 [==============================] - 5732s 4s/step - loss: 0.6405 - accuracy: 0.6204 - val_loss: 0.3249 - val_accuracy: 0.8820
Epoch 2/20
1366/1366 [==============================] - 493s 361ms/step - loss: 0.2001 - accuracy: 0.9372 - val_loss: 0.2237 - val_accuracy: 0.9196
Epoch 3/20
1366/1366 [==============================] - 487s 356ms/step - loss: 0.1642 - accuracy: 0.9505 - val_loss: 0.2272 - val_accuracy: 0.9177
Epoch 4/20
1366/1366 [==============================] - 488s 358ms/step - loss: 0.1496 - accuracy: 0.9541 - val_loss: 0.2037 - val_accuracy: 0.9422
Epoch 5/20
1366/1366 [==============================] - 486s 356ms/step - loss: 0.1457 - accuracy: 0.9548 - val_loss: 0.2014 - val_accuracy: 0.9393
Epoch 6/20
1366/1366 [==============================] - 482s 353ms/step - loss: 0.1364 - accuracy: 0.9570 - val_loss: 0.1981 - val_accuracy: 0.9363
Epoch 7/20
1366/1366 [==============================] - 487s 356ms/step - loss: 0.1332 - accuracy: 0.9579 - val_lo

In [48]:
model.save("malaria_model_cnn.h5")